## Import Statements

In [1]:
# math stuff
from math import floor
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# TensorFlow
import tensorflow as tf

# suppress numpy scientific e notation
np.set_printoptions(suppress=True)

/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


## TensorFlow Version

In [2]:
# tensorflow version
print(tf.__version__)

1.5.0


## Create Dummy Data Set

In [3]:
# create dummy data similar to what my database query returns (several features over time)
sequence = np.float64(np.arange(20000).reshape((5000, 4)))

## Define Parameters and Hyperparameters

In [4]:
# sequence variables
observation_length = 100 # will cause encoder lstm cells to unroll 'observation_length' times
prediction_length = 10 # will cause decoder lstm cells to unroll 'prediction_length' times
stride = 2 # window for batch generation will slide by 'stride' time steps
combined_length = observation_length + prediction_length
total_sequence_length = len(sequence)
train_sequence_length = floor(total_sequence_length * 0.6)
val_sequence_length = floor(total_sequence_length * 0.4)

# compute number of batches to emit
num_of_train_seq_batches = floor((train_sequence_length - combined_length) / stride)
num_of_val_seq_batches = floor((val_sequence_length - combined_length) / stride)
print("# of batches in training sequence:", num_of_train_seq_batches)
print("# of batches in validation sequence:", num_of_val_seq_batches)

# number of features going into the encoder
features_enc_inp = len(sequence[1])

# number of features of the target sequence
features_dec_inp = 1
features_dec_exp_out = features_dec_inp

# number of batches used in each iteration
batch_size = 3
print("batch size:", batch_size)

# we're using batch major for now
time_major = False

# defining layers and number of units for basic lstm cells
enc_num_cells = 1 # how many lstm cells are we using
enc_num_units = 50 # how many lstm units, or commonly known as hidden dimensions/neurons, shall each lstm cell have
dec_num_cells = enc_num_cells
dec_num_units = enc_num_units

# optimizer variables
learning_rate = 0.0003
lr_decay = 0.95
momentum = 0.5
lambda_l2_reg = 0.02

# dropout
dropout_keep_prob = 0.5

# training parameters
num_batches_train = floor(num_of_train_seq_batches / batch_size)
num_batches_val = floor(num_of_val_seq_batches / batch_size)
num_epochs_train = 100
early_stop_limit = 100
print("# of training steps:", num_batches_train)
print("# of validation steps:", num_batches_val)
train_to_val_ratio = floor(num_batches_train / num_batches_val)
print("run validation op every:", train_to_val_ratio, "training steps")
num_batches_train = train_to_val_ratio * num_batches_val
print("new # of batches for training sequence:", num_batches_train)
print("early stop after:", early_stop_limit, "validation iterations without improvement")

# of batches in training sequence: 1445
# of batches in validation sequence: 945
batch size: 3
# of training steps: 481
# of validation steps: 315
run validation op every: 1 training steps
new # of batches for training sequence: 315
early stop after: 100 validation iterations without improvement


## Create Generator for Training Data Set

In [5]:
# define generator function for training data
def gen_train():
    train_sequence = sequence[0:train_sequence_length]
    
    # transform and emit data in batches
    for i in range(0, num_of_train_seq_batches * stride, stride):
        result = np.array(train_sequence[i:i + combined_length])
        scaler = MinMaxScaler()
        result = scaler.fit_transform(result, y=None)
        
        # flip array upside down as data is ordered by date desc
        result_flipped = np.flipud(result)
        
        # yield results
        yield result

## Create Generator for Validation Data Set

In [6]:
# define generator function for validation data
def gen_val():
    val_sequence = sequence[train_sequence_length:train_sequence_length+val_sequence_length]
    
    # transform and emit data in batches
    for i in range(0, num_of_val_seq_batches * stride, stride):
        result = np.array(val_sequence[i:i + combined_length])
        scaler = MinMaxScaler()
        result = scaler.fit_transform(result, y=None)
        
        # flip array upside down as data is ordered by date desc
        result_flipped = np.flipud(result)
        
        # yield results
        yield result

## Reset Default TensorFlow Graph and Create Sequence Length Tensors

In [7]:
# reset default tf graph
tf.reset_default_graph()

# length of input and output
seq_length_inp = tf.fill([batch_size], observation_length)
seq_length_out = tf.fill([batch_size], prediction_length)

## Create TensorFlow Training Input and Output

In [8]:
# create placeholder for training next element
train_next_element = tf.placeholder(
    tf.float32, 
    shape=(combined_length, features_enc_inp), 
    name="next_train_element_from_generator")

# create training dataset from generator
train_dataset = tf.data.Dataset.from_generator(
    gen_train,
    (tf.float32),
    (tf.TensorShape([combined_length, features_enc_inp]))
    )

# prefetch 'number of batches' training sequences of data
train_prefetched = train_dataset.prefetch(num_of_train_seq_batches)

# shuffle training batches
train_buffer_size = tf.constant(
    num_of_train_seq_batches,
    dtype=tf.int64)

train_shuffled = train_prefetched.shuffle(
    train_buffer_size,
    seed=None
)

# batch training batches together 'batch size' times
train_batched = train_shuffled.batch(batch_size)

# create initializable training iterator
train_iterator = train_batched.make_initializable_iterator()
train_next_element = train_iterator.get_next()

# create training encoder input slice 
# from [first batch, first time step in observation sequence, first feature]
# to [last batch, last time step in observation sequence, last feature]
train_enc_inp = tf.slice(
    train_next_element,
    [0, 0, 0],
    [batch_size, observation_length, features_enc_inp])

# create training decoder input slice 
# from [first batch, first time step in prediction sequence, first feature]
# to [last batch, last time step in prediction sequence, first feature]
train_dec_inp = tf.slice(
    train_next_element,
    [0, observation_length, 0],
    [batch_size, prediction_length, features_dec_inp])

# create training decoder expected output slice 
# from [first batch, first time step in prediction sequence, first feature]
# to [last batch, last time step in prediction sequence, first feature]
train_dec_exp_out = tf.slice(
    train_next_element,
    [0, observation_length, 0],
    [batch_size, prediction_length, features_dec_exp_out])

## Create TensorFlow Validation Input and Output

In [9]:
# create placeholder for validation next element
val_next_element = tf.placeholder(
    tf.float32, 
    shape=(combined_length, features_enc_inp), 
    name="next_val_element_from_generator")

# create validation dataset from generator
val_dataset = tf.data.Dataset.from_generator(
    gen_val,
    (tf.float32),
    (tf.TensorShape([combined_length, features_enc_inp]))
    )

# prefetch 'number of batches' validation sequences of data
val_prefetched = val_dataset.prefetch(num_of_val_seq_batches)

# shuffle validation batches
val_buffer_size = tf.constant(
    num_of_val_seq_batches,
    dtype=tf.int64)

val_shuffled = val_prefetched.shuffle(
    val_buffer_size,
    seed=None
)

# batch validation batches together 'batch size' times
val_batched = val_shuffled.batch(batch_size)

# create initializable validation iterator
val_iterator = val_batched.make_initializable_iterator()
val_next_element = val_iterator.get_next()

# create validation encoder input slice 
# from [first batch, first time step in observation sequence, first feature]
# to [last batch, last time step in observation sequence, last feature]
val_enc_inp = tf.slice(
    val_next_element,
    [0, 0, 0],
    [batch_size, observation_length, features_enc_inp])

# create validation decoder input slice 
# from [first batch, first time step in prediction sequence, first feature]
# to [last batch, last time step in prediction sequence, first feature]
val_dec_inp = tf.slice(
    val_next_element,
    [0, observation_length, 0],
    [batch_size, prediction_length, features_dec_inp])

# create validation decoder expected output slice 
# from [first batch, first time step in prediction sequence, first feature]
# to [last batch, last time step in prediction sequence, first feature]
val_dec_exp_out = tf.slice(
    val_next_element,
    [0, observation_length, 0],
    [batch_size, prediction_length, features_dec_exp_out])

## Create Encoder and Decoder Cells

In [10]:
with tf.variable_scope('Seq2seq', reuse = tf.AUTO_REUSE):

    # create encoder cells
    enc_cells = []
    for i in range(enc_num_cells):
        enc_cells.append(tf.nn.rnn_cell.BasicLSTMCell(enc_num_units))
    enc_cell = tf.nn.rnn_cell.MultiRNNCell(enc_cells)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob = dropout_keep_prob)

    # create decoder cells
    dec_cells = []
    for i in range(dec_num_cells):
        dec_cells.append(tf.nn.rnn_cell.BasicLSTMCell(dec_num_units))
    dec_cell = tf.nn.rnn_cell.MultiRNNCell(dec_cells)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob = dropout_keep_prob)

## Define Training Model

In [11]:
with tf.variable_scope('Seq2seq', reuse = tf.AUTO_REUSE): 

    # define training encoder
    train_enc_out, train_enc_state = tf.nn.dynamic_rnn(
        enc_cell, 
        train_enc_inp,
        dtype = tf.float32,
        sequence_length = seq_length_inp,
        time_major = time_major)

    # define training training helper
    train_training_helper = tf.contrib.seq2seq.TrainingHelper(
        train_dec_inp, 
        seq_length_out, 
        time_major = time_major)

    # define training decoder
    train_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, 
        train_training_helper, 
        train_enc_state)

    # define training dynamic decoding
    train_dec_out, train_dec_state, train_dec_out_seq_length = tf.contrib.seq2seq.dynamic_decode(
        train_decoder, 
        output_time_major = time_major)

    # extract training logits from decoder output
    train_dec_out_logits = train_dec_out.rnn_output

    # training dense layer to reduce output to 'features_dec_exp_out' feature(s)
    train_output_dense = tf.layers.dense(
        train_dec_out_logits,
        features_dec_exp_out)

    # training loss function
    train_loss = tf.reduce_mean(tf.nn.l2_loss(train_output_dense - train_dec_exp_out))

    # l2 regularization using all variables except for bias and batch_resample
    l2 = lambda_l2_reg * sum(
        tf.nn.l2_loss(tf_var)
            for tf_var in tf.trainable_variables()
            if not ("noreg" in tf_var.name or "Bias" in tf_var.name)
    )
    train_loss += l2

    # optimizer type and variables
    optimizer = tf.train.RMSPropOptimizer(
        learning_rate,
        decay = lr_decay,
        momentum = momentum,
        epsilon = 1e-10)

    # train operation
    train_op = optimizer.minimize(train_loss)

## Define Validation Model

In [12]:
with tf.variable_scope('Seq2seq', reuse = tf.AUTO_REUSE):    
    
    # define validation encoder
    val_enc_out, val_enc_state = tf.nn.dynamic_rnn(
        enc_cell, 
        val_enc_inp,
        dtype = tf.float32,
        sequence_length = seq_length_inp,
        time_major = time_major)
    
    # define validation training helper
    val_training_helper = tf.contrib.seq2seq.TrainingHelper(
        val_dec_inp, 
        seq_length_out, 
        time_major = time_major)
    
     # define validation decoder
    val_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, 
        val_training_helper, 
        val_enc_state)
    
    # define validation dynamic decoding
    val_dec_out, val_dec_state, val_dec_out_seq_length = tf.contrib.seq2seq.dynamic_decode(
        val_decoder, 
        output_time_major = time_major)
    
    # extract validation logits from decoder output
    val_dec_out_logits = val_dec_out.rnn_output
    
    # validation dense layer to reduce output to 'features_dec_exp_out' feature(s)
    val_output_dense = tf.layers.dense(
        val_dec_out_logits,
        features_dec_exp_out)
    
    # validation loss function
    val_loss = tf.reduce_mean(tf.nn.l2_loss(val_output_dense - val_dec_exp_out))
    
    # l2 regularization using all variables except for bias and batch_resample
    l2 = lambda_l2_reg * sum(
        tf.nn.l2_loss(tf_var)
            for tf_var in tf.trainable_variables()
            if not ("noreg" in tf_var.name or "Bias" in tf_var.name)
    )
    val_loss += l2

## Check Trainable Variables

In [ ]:
for tf_var in tf.trainable_variables():
    print(tf_var)

<tf.Variable 'Seq2seq/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(54, 200) dtype=float32_ref>
<tf.Variable 'Seq2seq/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'Seq2seq/decoder/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(51, 200) dtype=float32_ref>
<tf.Variable 'Seq2seq/decoder/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'Seq2seq/dense/kernel:0' shape=(50, 1) dtype=float32_ref>
<tf.Variable 'Seq2seq/dense/bias:0' shape=(1,) dtype=float32_ref>


## Run Training and Validation Session

In [ ]:
# define session
sess = tf.InteractiveSession()

# initialize global variables / check for uninitialized variables
sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

# check for unitialized variables 
# print(sess.run(tf.report_uninitialized_variables())

lowest_val_loss = 100.0
early_stop_counter = 0

# run training and validation
print("epoch batch training_loss validation_loss lowest_val_loss early_stop_counter")
for epoch in range(num_epochs_train):
    sess.run([train_iterator.initializer, val_iterator.initializer])
    for batch in range(num_batches_train):
        if early_stop_counter <= early_stop_limit:
            try:
                train_results = sess.run([train_op, train_loss])
                if batch % train_to_val_ratio == 0:
                    try:
                        val_results = sess.run(val_loss)
                    except tf.errors.OutOfRangeError:
                        print("end of validation dataset")
                    if val_results < lowest_val_loss:
                        lowest_val_loss = val_results
                        early_stop_counter = 0
                    else:
                        early_stop_counter += 1
                    print(epoch, batch, train_results[1], val_results, lowest_val_loss, early_stop_counter)
                else:
                    pass
            except tf.errors.OutOfRangeError:
                print("end of training dataset")
        else:
            pass

epoch batch training_loss validation_loss lowest_val_loss early_stop_counter
0 0 10.188868 10.73506 10.73506 0
0 1 10.9222975 9.408237 9.408237 0
0 2 10.224509 8.082983 8.082983 0
0 3 8.0427475 8.664698 8.082983 1
0 4 8.139303 7.4073257 7.4073257 0
0 5 7.850829 6.485696 6.485696 0
0 6 6.912312 6.556961 6.485696 1
0 7 6.2842894 5.348409 5.348409 0
0 8 5.981259 5.047156 5.047156 0
0 9 5.615658 4.9587755 4.9587755 0
0 10 4.8780704 4.2950454 4.2950454 0
0 11 4.839566 4.481906 4.2950454 1
0 12 4.3799343 4.3481326 4.2950454 2
0 13 4.7330017 3.6544497 3.6544497 0
0 14 3.6890442 4.3452673 3.6544497 1
0 15 4.5670166 3.7635574 3.6544497 2
0 16 3.3831673 3.5850058 3.5850058 0
0 17 3.6612422 3.5139294 3.5139294 0
0 18 3.8804169 3.8983788 3.5139294 1
0 19 2.7191615 2.8338025 2.8338025 0
0 20 3.8989286 2.9804356 2.8338025 1
0 21 3.3468904 2.978497 2.8338025 2
0 22 2.8058844 3.262672 2.8338025 3
0 23 3.1299977 2.7962072 2.7962072 0
0 24 2.5803943 2.8808432 2.7962072 1
0 25 2.9249802 2.7421834 2.74218

0 223 1.7916329 1.7539783 1.5725737 2
0 224 1.6427907 1.6166761 1.5725737 3
0 225 1.605714 1.5850879 1.5725737 4
0 226 1.6078773 1.61682 1.5725737 5
0 227 1.6755089 1.7841877 1.5725737 6
0 228 1.6689626 1.5503935 1.5503935 0
0 229 1.6048154 1.6307768 1.5503935 1
0 230 1.7635236 1.6554654 1.5503935 2
0 231 1.5579989 1.7227505 1.5503935 3
0 232 1.5725133 1.7547075 1.5503935 4
0 233 1.5535668 1.6199689 1.5503935 5
0 234 1.5195019 1.6926794 1.5503935 6
0 235 1.4929392 1.6372939 1.5503935 7
0 236 1.449546 1.4715238 1.4715238 0
0 237 1.5836613 1.4832002 1.4715238 1
0 238 1.4972197 1.6420125 1.4715238 2
0 239 1.64726 1.5229969 1.4715238 3
0 240 1.5300462 1.5404892 1.4715238 4
0 241 1.5622987 1.4986775 1.4715238 5
0 242 1.4351223 1.5605263 1.4715238 6
0 243 1.5106213 1.4404945 1.4404945 0
0 244 1.5366594 1.4223449 1.4223449 0
0 245 1.570168 1.4998376 1.4223449 1
0 246 1.4436326 1.4383833 1.4223449 2
0 247 1.4239254 1.4326838 1.4223449 3
0 248 1.422807 1.504662 1.4223449 4
0 249 1.3927897 1.502